# Feature Extraction Notebook

Create Ready-to-use Sklearn Transformers

## Table of Contents:
1. [Imports](#imports)
1. [Constants](#constants)
1. [Geo Level Feature Extraction](#geo-level-feature-extraction)  
    1. [Geo Level ID: Dimensionality Reduction](#geo-dim-reduction)  
        1. [Model and Dataset Definition](#geo-dim-reduction-model-def)
        1. [Training](#geo-dim-reduction-train)
    1. [Geo Level ID: Guess Geo 3 Roll up to Geo 1 and 2](#geo3-rollup)
        1. [Model and Dataset Definition](#geo-rollup-model-def)
        1. [Training](#geo-rollup-train)
1. [All Features](#all-feature-extraction)
    1. [Simple Autoencoder](#all-feature-ae)

## Imports <a class="anchor" id="imports"></a>

In [5]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.base import BaseEstimator, TransformerMixin, ClassNamePrefixFeaturesOutMixin
from typing import Tuple, Any
from tqdm import tqdm
from pathlib import Path
from os import PathLike

## Constants <a id="constants"></a>

In [2]:
TRAINING_FEATURES_PATH="D:/ml_competitions/gorkha_earthquake/data/raw/train_values.csv"
TRAINING_LABELS_PATH="D:/ml_competitions/gorkha_earthquake/data/raw/train_labels.csv"
TEST_FEATURES_PATH="D:/ml_competitions/gorkha_earthquake/data/raw/test_values.csv"
SUBMISSION_FORMAT_PATH="D:/ml_competitions/gorkha_earthquake/data/raw/submission_format.csv"

SUBMISSION_DIR="D:/ml_competitions/gorkha_earthquake/submissions"

## Load Data

In [3]:
features_df         = pd.read_csv(TRAINING_FEATURES_PATH,   index_col=0)
labels_df           = pd.read_csv(TRAINING_LABELS_PATH,     index_col=0) - 1
test_features_df    = pd.read_csv(TEST_FEATURES_PATH,       index_col=0)

## Geo Level Feature Extraction <a class="anchor" id="geo-level-feature-extraction"></a>

Label Encode <a id="geo-dim-reduction-le"></a>

In [8]:
import pickle
from sklearn.preprocessing import LabelEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

df = pd.concat([features_df, test_features_df])

le1 = LabelEncoder().fit(df['geo_level_1_id'])
le2 = LabelEncoder().fit(df['geo_level_2_id'])
le3 = LabelEncoder().fit(df['geo_level_3_id'])

eval = le3.transform(test_features_df['geo_level_3_id'])

# Save All Label Encoders
with open(Path.cwd().parent / 'models' / 'geo-lv-1-label-encoder.pickle', 'wb') as f:
    pickle.dump(le1, f)
with open(Path.cwd().parent / 'models' / 'geo-lv-2-label-encoder.pickle', 'wb') as f:
    pickle.dump(le2, f)
with open(Path.cwd().parent / 'models' / 'geo-lv-3-label-encoder.pickle', 'wb') as f:
    pickle.dump(le3, f)

del le1, le2, le3

# Load All Label Encoders
with open(Path.cwd().parent / 'models' / 'geo-lv-1-label-encoder.pickle', 'rb') as f:
    le1 = pickle.load(f)
with open(Path.cwd().parent / 'models' / 'geo-lv-2-label-encoder.pickle', 'rb') as f:
    le2 = pickle.load(f)
with open(Path.cwd().parent / 'models' / 'geo-lv-3-label-encoder.pickle', 'rb') as f:
    le3 = pickle.load(f)

# Prepare Transformers
geo_lv1_le = FunctionTransformer(
    func=lambda x: np.array(le1.transform(x.values.ravel())).reshape(-1, 1),
    feature_names_out='one-to-one'
)

geo_lv2_le = FunctionTransformer(
    func=lambda x: np.array(le2.transform(x.values.ravel())).reshape(-1, 1), 
    feature_names_out='one-to-one'
)

geo_lv3_le = FunctionTransformer(
    func=lambda x: np.array(le3.transform(x.values.ravel())).reshape(-1, 1), 
    feature_names_out='one-to-one'
)

In [9]:
# Demonstration on how to use the above in a column transformer
preprocessor = ColumnTransformer([
    ('geo1_le', geo_lv1_le, ['geo_level_1_id']),
    ('geo2_le', geo_lv2_le, ['geo_level_2_id']),
    ('geo3_le', geo_lv3_le, ['geo_level_3_id']),
], remainder='drop', verbose_feature_names_out=False)

preprocessor.set_output(transform='pandas')

pipeline = Pipeline([
    ('prep', preprocessor),
])

In [10]:
df = pd.concat([features_df, test_features_df])
df: pd.DataFrame = preprocessor.fit_transform(df) # type: ignore

# Get max ID in Geo Level 1
max_geo_lv1_id = df['geo_level_1_id'].max() + 1
# Get max ID in Geo Level 2
max_geo_lv2_id = df['geo_level_2_id'].max() + 1
# Get max ID in Geo Level 3
max_geo_lv3_id = df['geo_level_3_id'].max() + 1

print(
    max_geo_lv1_id,
    max_geo_lv2_id,
    max_geo_lv3_id
)

31 1418 11861


### Geo Level ID: Dimensionality Reduction <a class="anchor" id="geo-dim-reduction"></a>

In [7]:
geo_lv1_counts = df['geo_level_1_id'].value_counts(normalize=True)
geo_lv1_weights = np.zeros(max_geo_lv1_id)
geo_lv1_weights[geo_lv1_counts.index] = 1.0 / geo_lv1_counts.to_numpy()

geo_lv2_counts = df['geo_level_2_id'].value_counts(normalize=True)
geo_lv2_weights = np.zeros(max_geo_lv2_id)
geo_lv2_weights[geo_lv2_counts.index] = 1.0 / geo_lv2_counts.to_numpy()

geo_lv3_counts = df['geo_level_3_id'].value_counts(normalize=True)
geo_lv3_weights = np.zeros(max_geo_lv3_id)
geo_lv3_weights[geo_lv3_counts.index] = 1.0 / geo_lv3_counts.to_numpy()

Define Dataset and Model <a id="geo-dim-reduction-model-def"></a>

In [8]:
# Create dataset for Autoencoder training
dataset = TensorDataset(
    torch.from_numpy(
        (
            df[['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']]
                .to_numpy()
        )
    ).type(torch.long),

    torch.from_numpy(
        (
            df[['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']]
                .to_numpy()
        )
    ).type(torch.long)
)


class DREncoder(torch.nn.Module):

    def __init__(self, 
                 latent_dim: int=16, 
                 geo_lv1_size: int=31, 
                 geo_lv2_size: int=1428,
                 geo_lv3_size: int=12568) -> None:
        super().__init__()
        self.geo_lv1_embedder = torch.nn.Embedding(geo_lv1_size, 16)
        self.geo_lv2_embedder = torch.nn.Embedding(geo_lv2_size, 128)
        self.geo_lv3_embedder = torch.nn.Embedding(geo_lv3_size, 128) 
        self.compressor = torch.nn.Linear(16+128+128, latent_dim)

    def forward(self, x):
        x_1 = self.geo_lv1_embedder(x[:, 0])
        x_2 = self.geo_lv2_embedder(x[:, 1])
        x_3 = self.geo_lv3_embedder(x[:, 2])
        x = torch.concat((x_1, x_2, x_3), dim=1)
        x = torch.nn.functional.relu(x)
        return self.compressor(x)


class DRDecoder(torch.nn.Module):
    
    def __init__(self,                  
                 latent_dim: int=16, 
                 geo_lv1_size: int=31, 
                 geo_lv2_size: int=1428,
                 geo_lv3_size: int=12568) -> None:
        super().__init__()
        self.geo_lv1_predictor = torch.nn.Linear(latent_dim, geo_lv1_size)
        self.geo_lv2_predictor = torch.nn.Linear(latent_dim, geo_lv2_size)
        self.geo_lv3_predictor = torch.nn.Linear(latent_dim, geo_lv3_size)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        x1 = self.geo_lv1_predictor(x)
        x2 = self.geo_lv2_predictor(x)
        x3 = self.geo_lv3_predictor(x)
        return x1, x2, x3


class DRAutoEncoder(torch.nn.Module):
    
    def __init__(self,
                 latent_dim: int=16, 
                 geo_lv1_size: int=31, 
                 geo_lv2_size: int=1428,
                 geo_lv3_size: int=12568) -> None:
        super().__init__()
        self.encoder = DREncoder(latent_dim, geo_lv1_size, geo_lv2_size, geo_lv3_size)
        self.decoder = DRDecoder(latent_dim, geo_lv1_size, geo_lv2_size, geo_lv3_size)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        x = self.encoder(x)
        x = torch.nn.functional.relu(x)
        x1, x2, x3 = self.decoder(x)
        return x1, x2, x3

Training <a id="geo-dim-reduction-train"></a>

In [14]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

num_epochs = 100
dataloader = DataLoader(dataset, 2*1024)
model = DRAutoEncoder(
    latent_dim=32, 
    geo_lv1_size=max_geo_lv1_id, 
    geo_lv2_size=max_geo_lv2_id, 
    geo_lv3_size=max_geo_lv3_id
).to(DEVICE)

has_weights = True
criterion_geo_lv1 = torch.nn.CrossEntropyLoss(
    (
        torch
            .from_numpy(geo_lv1_weights)
            .type(torch.float)
            .to(DEVICE)
    ) if has_weights else None
)
criterion_geo_lv2 = torch.nn.CrossEntropyLoss(
    (
        torch
            .from_numpy(geo_lv2_weights)
            .type(torch.float)
            .to(DEVICE)
    ) if has_weights else None
)
criterion_geo_lv3 = torch.nn.CrossEntropyLoss(
    (
        torch
            .from_numpy(geo_lv3_weights)
            .type(torch.float)
            .to(DEVICE)
    ) if has_weights else None
)
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(num_epochs):
    print(f"EPOCH {epoch+1}")
    model.train()
    training_loss = 0.0
    for x, y in tqdm(dataloader, desc="training"):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        a = model(x)

        loss: torch.Tensor = (
            criterion_geo_lv1(a[0], y[:, 0])
            + criterion_geo_lv2(a[1], y[:, 1])
            + criterion_geo_lv3(a[2], y[:, 2])
        ) / 3

        training_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"{training_loss = }")

    # model.eval()
    # validation_loss = 0.0
    # for x, y in tqdm(dataloader, desc="validation"):
    #     with torch.no_grad():
    #         x = x.to(DEVICE)
    #         y = y.to(DEVICE)
    #         a = model(x)

    #         loss: torch.Tensor = (
    #             criterion_geo_lv1(a[0], y[:, 0])
    #             + criterion_geo_lv2(a[1], y[:, 1])
    #             + criterion_geo_lv3(a[2], y[:, 2])
    #         ) / 3

    #         validation_loss += loss.item()

    # print(f"{validation_loss = }")


EPOCH 1


training: 100%|██████████| 170/170 [00:05<00:00, 29.73it/s]


training_loss = 1009.220043182373
EPOCH 2


training: 100%|██████████| 170/170 [00:04<00:00, 36.45it/s]


training_loss = 471.63681995868683
EPOCH 3


...

training_loss = 0.0002598527535155881
EPOCH 99


training: 100%|██████████| 170/170 [00:05<00:00, 31.31it/s]


training_loss = 0.00023767101538396673
EPOCH 100


training: 100%|██████████| 170/170 [00:05<00:00, 30.79it/s]

training_loss = 0.0002173561360336862


In [15]:
torch.save(model.encoder.state_dict(), Path.cwd().parent / 'models' / 'dim-reduction-32')

Sklearn Transformer

In [ ]:
class DREncoder(torch.nn.Module):

    def __init__(self, 
                 latent_dim: int=16, 
                 geo_lv1_size: int=31, 
                 geo_lv2_size: int=1418,
                 geo_lv3_size: int=11861) -> None:
        super().__init__()
        self.geo_lv1_embedder = torch.nn.Embedding(geo_lv1_size, 16)
        self.geo_lv2_embedder = torch.nn.Embedding(geo_lv2_size, 128)
        self.geo_lv3_embedder = torch.nn.Embedding(geo_lv3_size, 128) 
        self.compressor = torch.nn.Linear(16+128+128, latent_dim)

    def forward(self, x):
        x_1 = self.geo_lv1_embedder(x[:, 0])
        x_2 = self.geo_lv2_embedder(x[:, 1])
        x_3 = self.geo_lv3_embedder(x[:, 2])
        x = torch.concat((x_1, x_2, x_3), dim=1)
        x = torch.nn.functional.relu(x)
        return self.compressor(x)


class GeoDimensionReduction(BaseEstimator, TransformerMixin, ClassNamePrefixFeaturesOutMixin):

    def __init__(
            self, 
            path: PathLike,
            latent_dim: int=16, 
            geo_lv1_size: int=31,
            geo_lv2_size: int=1418,
            geo_lv3_size: int=11861) -> None:
        super().__init__()
        self.path = path
        self.model = DREncoder(
            latent_dim, 
            geo_lv1_size,
            geo_lv2_size,
            geo_lv3_size
        )
        self.latent_dim = latent_dim
        self.geo_lv1_size = geo_lv1_size
        self.geo_lv2_size = geo_lv2_size
        self.geo_lv3_size = geo_lv3_size
        self.model.load_state_dict(torch.load(path))

    def fit(self, X: pd.DataFrame, y=None, *args, **kwargs):
        return self

    def transform(self, X: pd.DataFrame, y=None, *args, **kwargs):
        # Convert pd to numpy
        X = X.values
        # Apply encoder
        self.model.eval()
        X = torch.from_numpy(X).type(torch.long) # type: ignore
        return self.model(X).detach().numpy()

In [ ]:
x = features_df

geo_dim_reduction_pipe = Pipeline([
    ('label_encoder', preprocessor),
    ('embedder', GeoDimensionReduction(path=Path.cwd().parent / 'models' / 'dim-reduction-16')),
])

c = ColumnTransformer([
    ('geo_dim_reduction', geo_dim_reduction_pipe, ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'])
], remainder='passthrough')
c.set_output(transform='pandas')
x_ = c.fit_transform(x)

### Geo Level ID: Guess Geo 3 Roll up to Geo 1 and 2 <a class="anchor" id="geo3-rollup"></a>

In [7]:
geo_lv1_counts = df['geo_level_1_id'].value_counts(normalize=True)
geo_lv1_weights = np.zeros(max_geo_lv1_id)
geo_lv1_weights[geo_lv1_counts.index] = 1.0 / geo_lv1_counts.to_numpy()

geo_lv2_counts = df['geo_level_2_id'].value_counts(normalize=True)
geo_lv2_weights = np.zeros(max_geo_lv2_id)
geo_lv2_weights[geo_lv2_counts.index] = 1.0 / geo_lv2_counts.to_numpy()

Dataset and Model Definition <a id="geo-rollup-model-def"></a>

In [8]:
# Create dataset for Autoencoder training
dataset = TensorDataset(
    torch.from_numpy(
        (
            df[['geo_level_3_id']]
                .to_numpy()
        )
    ).type(torch.long),

    torch.from_numpy(
        (
            df[['geo_level_1_id', 'geo_level_2_id']]
                .to_numpy()
        )
    ).type(torch.long)
)


class RollUpGeoLv3Encoder(torch.nn.Module):

    def __init__(self, 
                 latent_dim: int=16, 
                 geo_lv3_size: int=11861) -> None:
        super().__init__()
        self.geo_lv3_embedder = torch.nn.Embedding(geo_lv3_size, 128)
        self.compressor = torch.nn.Linear(128, latent_dim)

    def forward(self, x):
        x = self.geo_lv3_embedder(x).squeeze(1)
        x = torch.nn.functional.relu(x)
        return self.compressor(x)


class RollUpGeoLv3Decoder(torch.nn.Module):
    
    def __init__(self,                  
                 latent_dim: int=16, 
                 geo_lv1_size: int=31, 
                 geo_lv2_size: int=1418) -> None:
        super().__init__()
        self.geo_lv1_predictor = torch.nn.Linear(latent_dim, geo_lv1_size)
        self.geo_lv2_predictor = torch.nn.Linear(latent_dim, geo_lv2_size)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        x1 = self.geo_lv1_predictor(x)
        x2 = self.geo_lv2_predictor(x)
        return x1, x2


class RollUpGeoLv3AutoEncoder(torch.nn.Module):
    
    def __init__(self,
                 latent_dim: int=16, 
                 geo_lv1_size: int=31, 
                 geo_lv2_size: int=1418,
                 geo_lv3_size: int=11861) -> None:
        super().__init__()
        self.encoder = RollUpGeoLv3Encoder(latent_dim, geo_lv3_size)
        self.decoder = RollUpGeoLv3Decoder(latent_dim, geo_lv1_size, geo_lv2_size)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        x = self.encoder(x)
        x = torch.nn.functional.relu(x)
        x1, x2 = self.decoder(x)
        return x1, x2

Training <a id="geo-rollup-train"></a>

In [9]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

num_epochs = 100
dataloader = DataLoader(dataset, 2*1024)
model = RollUpGeoLv3AutoEncoder().to(DEVICE)

has_weights = True
criterion_geo_lv1 = torch.nn.CrossEntropyLoss(
    (
        torch
            .from_numpy(geo_lv1_weights)
            .type(torch.float)
            .to(DEVICE)
    ) if has_weights else None
)
criterion_geo_lv2 = torch.nn.CrossEntropyLoss(
    (
        torch
            .from_numpy(geo_lv2_weights)
            .type(torch.float)
            .to(DEVICE)
    ) if has_weights else None
)

optimizer = torch.optim.Adam(model.parameters())

for epoch in range(num_epochs):
    print(f"EPOCH {epoch+1}")
    model.train()
    training_loss = 0.0
    for x, y in tqdm(dataloader, desc="training"):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        a = model(x)

        loss: torch.Tensor = (
            criterion_geo_lv1(a[0], y[:, 0])
            + criterion_geo_lv2(a[1], y[:, 1])
        ) / 2

        training_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    training_loss /= len(dataloader)
    print(f"{training_loss = }")

    # model.eval()
    # validation_loss = 0.0
    # for x, y in tqdm(dataloader, desc="validation"):
    #     with torch.no_grad():
    #         x = x.to(DEVICE)
    #         y = y.to(DEVICE)
    #         a = model(x)

    #         loss: torch.Tensor = (
    #             criterion_geo_lv1(a[0], y[:, 0])
    #             + criterion_geo_lv2(a[1], y[:, 1])
    #         ) / 2

    #         validation_loss += loss.item()
    
    # validation_loss /= len(dataloader)
    # print(f"{validation_loss = }")


EPOCH 1


training: 100%|██████████| 170/170 [00:31<00:00,  5.44it/s]


training_loss = 901.0604166984558


validation: 100%|██████████| 170/170 [00:04<00:00, 34.13it/s]


validation_loss = 880.2212710380554
EPOCH 2


...

validation_loss = 0.001999931476348138
EPOCH 100


training: 100%|██████████| 170/170 [00:03<00:00, 49.87it/s]


training_loss = 0.0020042338369421486


validation: 100%|██████████| 170/170 [00:03<00:00, 49.64it/s]

validation_loss = 0.0018194816748291487


In [10]:
torch.save(model.encoder.state_dict(), Path.cwd().parent / 'models' / 'geo3-rollup-16')

sklearn Transformer

In [6]:
class RollUpGeoLv3Encoder(torch.nn.Module):

    def __init__(self, 
                 latent_dim: int=16, 
                 geo_lv3_size: int=11861) -> None:
        super().__init__()
        self.geo_lv3_embedder = torch.nn.Embedding(geo_lv3_size, 128)
        self.compressor = torch.nn.Linear(128, latent_dim)

    def forward(self, x):
        x = self.geo_lv3_embedder(x).squeeze(1)
        x = torch.nn.functional.relu(x)
        return self.compressor(x)


class GeoLv3Rollup(BaseEstimator, TransformerMixin, ClassNamePrefixFeaturesOutMixin):

    def __init__(
            self, 
            path: PathLike,
            latent_dim: int=16, 
            geo_lv3_size: int=11861) -> None:
        super().__init__()
        self.path = path
        self.model = RollUpGeoLv3Encoder(
            latent_dim, 
            geo_lv3_size
        )
        self.latent_dim = latent_dim
        self.geo_lv3_size = geo_lv3_size
        self.model.load_state_dict(torch.load(path))

    def fit(self, X: pd.DataFrame, y=None, *args, **kwargs):
        return self

    def transform(self, X: pd.DataFrame, y=None, *args, **kwargs):
        # Convert pd to numpy
        if isinstance(X, pd.DataFrame):
            X = X.values # type: ignore
        # Apply encoder
        self.model.eval()
        X = torch.from_numpy(X).type(torch.long) # type: ignore
        return self.model(X).detach().numpy()

In [21]:
x = features_df

# Demonstration on how to use the above in a column transformer
geo3_rollup_preprocessor = ColumnTransformer([
        ('geo3_le', geo_lv3_le, ['geo_level_3_id']),
    ], 
    remainder='drop', 
    verbose_feature_names_out=False,
).set_output(transform='pandas')

geo_rollup_pipe = Pipeline([
    ('label_encoder', geo3_rollup_preprocessor),
    ('embedder', GeoLv3Rollup(path=Path.cwd().parent / 'models' / 'geo3-rollup-16')),
])

c = ColumnTransformer([
    ('geo_rollup', geo_rollup_pipe, ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'])
], remainder='drop')
c.set_output(transform='pandas')

x_ = c.fit_transform(features_df)